In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode

In [2]:
import qualified Data.Attoparsec.ByteString.Char8 as C

parseLine = do
    C.string "Step "
    x <- C.anyChar
    C.string " must be finished before step "
    y <- C.anyChar
    C.string " can begin."
    pure (x,y)

input <- dayLines 7

pairs = map (parsed parseLine) input

In [3]:
import qualified Data.Set as Set

import qualified Data.Map.Strict as Map
import Data.List (foldl')

import Data.Tuple (swap)
import Data.List

swapped = map swap pairs

graph = foldl' (\m (k,v) -> Map.insertWith (++) k v m) Map.empty $ map (\(k,v) -> (k,[v])) swapped

initial = foldl' (\m c -> if Map.member c m then m else Map.insert c [] m) graph ['A'..'Z']

iter :: Map.Map Char String -> String -> String -> String
iter graph execute output
    | Map.null graph = output
    | otherwise = let
        (ready, blocked) = Map.partition (==[]) graph
        candidates = Map.keys ready
        (x:execute') = sort (candidates ++ execute)
        output' = x:output
        graph' = Map.map (delete x) blocked
        in iter graph' execute' output'

reverse $ iter initial [] []

"GKPTSLUXBIJMNCADFOVHEWYQRZ"

In [4]:
mapping = Map.fromList $ zip ['A'..'Z'] [1..26]

taskTime c = 60 + (mapping Map.! c)

data Task = Task Char Int deriving (Eq, Show)

type Workers = [Maybe Task]

work :: Task -> Task
work t@(Task _ 0) = t
work (Task c n) = Task c (n-1)

assignWork :: Workers -> String -> (Workers, String)
assignWork [] ts = ([], ts)
assignWork ws [] = (ws,[])
assignWork (w@(Just _):ws) ts = let
    (ws',ts') = assignWork ws ts
    in (w:ws',ts')
assignWork (Nothing:ws) (t:ts) = let
    (ws',ts') = assignWork ws ts
    in ((Just (Task t (taskTime t))):ws',ts')

finished (Just (Task _ 0)) = True
finished _ = False

unTask (Task t 0) = t

debrief (Just (Task _ 0)) = Nothing
debrief t = t

iter' :: Map.Map Char String -> String -> Workers -> Int -> Int
iter' graph tasks workers elapsed
    | Map.null graph && workers == [Nothing, Nothing, Nothing, Nothing, Nothing] = elapsed
    | otherwise = let
        (ready, blocked) = Map.partition (==[]) graph
        candidates = Map.keys ready
        (workers', tasks') = assignWork workers $ sort (candidates ++ tasks)
        workers'' = map (fmap work) workers'
        Just done = fmap (map unTask) $ sequenceA $ filter finished workers''
        workers''' = map debrief workers''
        graph' = Map.map (\\ done) blocked
        elapsed' = elapsed + 1
        in iter' graph' tasks' workers''' elapsed'

iter' initial [] [Nothing, Nothing, Nothing, Nothing, Nothing] 0

920